# Module 10 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [482]:
from unification import parse, unification
from copy import deepcopy

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

### <a id="parse_simple"></a> parse_simple

Formal Parameters:

**lst** A list of s-expressions -- The start state or goal state

**return** None

**Modifies** lst

Parses every element of lst, so that it can by used by unification

In [483]:
def parse_simple(lst):
    for i in range(len(lst)):
        lst[i] = parse(lst[i])

### <a id="parse_actions"></a> parse_actions

Formal Parameters:

**actions** The actions dictionary -- A dictionary containing all the actions, their preconditions, and postconditions.

**return** None

**Modifies** actions

Parses every action in actions, so that it can by used by unification

In [484]:
def parse_actions(actions):
    for action in actions.keys():
        actions[action]["action"] = parse(actions[action]["action"])
        for i in range(len(actions[action]["conditions"])):
            actions[action]["conditions"][i] = parse(actions[action]["conditions"][i])
        for i in range(len(actions[action]["add"])):
            actions[action]["add"][i] = parse(actions[action]["add"][i])
        for i in range(len(actions[action]["delete"])):
            actions[action]["delete"][i] = parse(actions[action]["delete"][i])

### <a id="build_possible_actions"></a> build_possible_actions

Formal Parameters:

**actions** The actions dictionary -- A dictionary containing all the actions, their preconditions, and postconditions.

**current_state** The current state --A list

**return** possible_actions-- a dictionary containing all possible actions in the current state, and all different ways of combining the variables that satisfy the preconditions of the actions.  May contain impossible actions, for the current state, but this is dealt with later, in [check_unify](#check_unify)


This dictionary is built so that it can be used in [possible_unification_iterator](#possible_unification_iterator).

In [485]:
def build_possible_actions(current_state,actions):
    possible_actions = {}
    for action in actions.keys():
        can_unify_action = True
        condition_unification_map = {}
        for condition in actions[action]["conditions"]:
            if len(condition) == 2:
                condition_unification_map[tuple(condition)] = []
            can_unify_particular = False
            for state in current_state:
                unify = unification(state,condition)
                if unify:
                    can_unify_particular = True
                    if len(condition) == 2:
                        condition_unification_map[tuple(condition)].append(state)
            if not can_unify_particular:
                can_unify_action = False
                break
        if can_unify_action:
            possible_actions[action] = condition_unification_map
            
        
    return possible_actions
    

In [486]:
halloween_current_state_test1 = [
    "(item Saw)",
    "(item Drill)",
    "(agent Me)",
    "(agent Other)",
    "(at Me Earth)",
    "(at Other Earth)"
    "(Place Earth)",
    "(Place Narnia)"
    "(Place Purgatory)"
]

halloween_actions_test1 = {
    "use_on_self": {
        "action": "(use_on_self ?agent ?item)",
        "conditions": [
            "(agent ?agent)",
            "(item ?item)",
            "(at ?agent Earth)"
        ],
        "add": [
            "(at ?agent Narnia)"
        ],
        "delete": [
            "(at ?agent Earth)"
        ]
    },
    "use_on_other": {
        "action": "(use_on_other ?agent1 ?agent2 ?item)",
        "conditions": [
            "(item ?item)",
            "(agent ?agent1)",
            "(agent ?agent2)",
            "(at ?agent1 Earth)",
            "(at ?agent2 Earth)"
        ],
        "add": [
            "(at ?agent2 Narnia)",
            "(at ?agent1 Purgatory)"
        ],
        "delete": [
            "(at ?agent2 Earth)",
            "(at ?agent1 Earth)"
        ]
    }
}
parse_simple(halloween_current_state_test1)
parse_actions(halloween_actions_test1)
x = build_possible_actions(halloween_current_state_test1,halloween_actions_test1)
print(x)

{'use_on_self': {('agent', '?agent'): [['agent', 'Me'], ['agent', 'Other']], ('item', '?item'): [['item', 'Saw'], ['item', 'Drill']]}, 'use_on_other': {('item', '?item'): [['item', 'Saw'], ['item', 'Drill']], ('agent', '?agent1'): [['agent', 'Me'], ['agent', 'Other']], ('agent', '?agent2'): [['agent', 'Me'], ['agent', 'Other']]}}


### <a id="possible_unification_iterator"></a> possible_unification_iterator

Formal Parameters:

**action** a dictionary that is one of the keys made in [build_possible_actions](#build_possible_actions)

**yield** a possible combination of constants that will unify with the variables in `actions[action]['action']`

This iterator yields every possible combination of constants that will unify. May yield impossible actions for the current state, but this is dealt with later, in [check_unify](#check_unify)


In [487]:
def possible_unification_iterator(action):
    choice_dict = {}
    curr_list = [0]*len(action.keys())
    choice_list = list(action.keys())
    for operand in action:
        choice_dict[operand] = action[operand][0]
    yield choice_dict
    curr = 0
    end = False
    while curr_list[len(curr_list)-1] != 0 or not end:
        curr_list[curr] = (curr_list[curr]+1)%len(action[choice_list[curr]])
        while curr_list[curr] == 0:
            curr+=1
            curr_list[curr] = (curr_list[curr]+1)%len(action[choice_list[curr]])
            if curr == len(curr_list)-1:
                end = True
                if curr_list[curr] == 0:
                    break
        curr = 0
        if curr_list == [0]*len(action.keys()):
            break
        for i in range(len(curr_list)):
            choice_dict[choice_list[i]] = action[choice_list[i]][curr_list[i]]
        yield choice_dict
        
    
    
    

In [488]:
for action in x.keys():
    z =possible_unification_iterator(x[action])
    for choice in z:
        print(action,choice)

use_on_self {('agent', '?agent'): ['agent', 'Me'], ('item', '?item'): ['item', 'Saw']}
use_on_self {('agent', '?agent'): ['agent', 'Other'], ('item', '?item'): ['item', 'Saw']}
use_on_self {('agent', '?agent'): ['agent', 'Me'], ('item', '?item'): ['item', 'Drill']}
use_on_self {('agent', '?agent'): ['agent', 'Other'], ('item', '?item'): ['item', 'Drill']}
use_on_other {('item', '?item'): ['item', 'Saw'], ('agent', '?agent1'): ['agent', 'Me'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Drill'], ('agent', '?agent1'): ['agent', 'Me'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Saw'], ('agent', '?agent1'): ['agent', 'Other'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Drill'], ('agent', '?agent1'): ['agent', 'Other'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Saw'], ('agent', '?agent1'): ['agent', 'Me'], ('agent', '?agent2'): ['agent',

### <a id="check_unify"></a> check_unify

Formal Parameters:

**check_state** the current state

**u** the suggested unification that is to be checked against check_state and action

**actions** the `actions` dictionary

**action** a string-- the key in the `actions` dictionary that is attempted to unify

**return** `True` or `False` -- whether the unification is successful or not.

This function ensures that the state satisfies the precondition for the suggested unification. Used in [make_new_current_state](#make_new_current_state)

In [489]:
def check_unify(check_state, u, actions, action):
    unify_check_list = deepcopy(actions[action]["conditions"])
    for stuff in check_state:
        for condition in unify_check_list:
            if unification(stuff, condition):
                match = []

                for word in condition:
                    addSimple = True
                    for k in u.keys():
                        if k[1] == word:
                            addSimple = False
                            match.append(u[k][1])
                            break
                    if addSimple:
                        match.append(word)
                if match in check_state:
                    unify_check_list.remove(condition)
                    continue
    if unify_check_list == []:
        return True
    return False

### <a id="make_new_current_state"></a> make_new_current_state

Formal Parameters:

**current_state** the current state

**u** the suggested unification that is to be checked against check_state and action

**actions** the `actions` dictionary

**action** a string-- the key in the `actions` dictionary that is attempted to unify

**return** `curr_state_copy`-- the new state after performing an action if the unification is successul, or `False` -- if not


This function ensures that the state satisfies the precondition for the suggested unification, then applies the effects of `actions[action]["add"]` and `actions[action]["add"]`. Used in [search](#search)

In [490]:
def make_new_current_state(current_state,u,actions,action):
    curr_state_copy = deepcopy(current_state)
    if not check_unify(curr_state_copy,u,actions,action):
        return False
    variable_to_constant_map = {}
    for v in u.keys():
        variable_to_constant_map[v[1]] = u[v][1]      
    for remove in actions[action]["delete"]:
        to_remove = []
        for term in remove:
            if term not in variable_to_constant_map.keys():
                to_remove.append(term)
            else:
                to_remove.append(variable_to_constant_map[term])
        try:
            curr_state_copy.remove(to_remove)
        except ValueError:
            pass
    for add in actions[action]["add"]:
        to_add = []
        for term in add:
            if term not in variable_to_constant_map.keys():
                to_add.append(term)
            else:
                to_add.append(variable_to_constant_map[term])
        if to_add not in curr_state_copy:
            curr_state_copy.append(to_add)
    return curr_state_copy
        

### <a id="seen"></a> seen

Formal Parameters:

**check_state** the state to check

**seen_states** a list of the states that have been seen

**return** `True` or `False`-- Whether or not `check_state` has been visited


A non-pythonic way of checking to see if a state has already been visited.  I completely forgot that I could simply use a `Set`.  Used to prevent inefficient plans and infinite loops/recursions in [search](#search).

In [491]:
def seen(check_state,seen_states):
    for state in seen_states:
        if len(state) != len(check_state):
            continue
        has_seen = True
        for s in state:
            if s not in check_state:
                has_seen = False
                break
        if has_seen:
            return True
    return False
            

### <a id="check_goal"></a> check_goal

Formal Parameters:

**check_state** the state to check

**goal** the goal

**return** `True` or `False`-- Whether or not `check_state` is the same as `goal`


A non-pythonic way of checking to see if a state is the goal.  I completely forgot that I could simply use a `Set`.  Used in [search](#search).

In [492]:
def check_goal(check_state,goal):

    if len(goal) != len(check_state):
        return False
    has_seen = True
    for s in goal:
        if s not in check_state:
            has_seen = False
            break
    if has_seen:
        return True
    return False

### <a id="build_back_action"></a> build_back_action

Formal Parameters:

**u** the unification dictionary of the relevant parameters in a particular action

**action** the string representation of the action

**actions** the actions dictionary

**return** `a`--The specific action taken on constants


Builds back the action taken so that it is in the required form for the `plan`. Used in [search](#search).

In [493]:
def build_back_action(u,action,actions):
    a = [action]
    for term in actions[action]["action"]:
        for k in u.keys():
            if k[1] == term:
                a.append(u[k][1])
    return a

### <a id="search"></a> search

Formal Parameters:

**current_state** a list representing the current state

**goal** the list representing the goal

**actions** the actions dictionary

**seen_states** a list of the states that have been seen

**plan** The list of actions taken so far, in order

**debug** A boolean -- Whether or not to include the intermediate states in plan

**return** The final **`plan`**, or `[]` if there can be no plan.


Called by [forward_planner](#forward_planner) to do the main work of the algorithm.  It performs the outer and inner loops described in the instructions.

In [494]:
def search(current_state, goal, actions, seen_states, plan, debug=False):
    possible_actions = build_possible_actions(current_state, actions)
    for action in possible_actions.keys():
        possible_unifications = possible_unification_iterator(possible_actions[action])
        for u in possible_unifications:
            check_state = make_new_current_state(current_state, u, actions, action)
            if not check_state:
                continue
            if check_goal(check_state,goal):
                plan.append(build_back_action(u,action,actions))
                if debug:
                    plan.append(check_state)
                return plan
            if not seen(check_state, seen_states):
                new_plan = deepcopy(plan)
                new_plan.append(build_back_action(u,action,actions))
                if debug:
                    new_plan.append(check_state)
                seen_states.append(check_state)
                next_search = search(check_state, goal, actions, seen_states, new_plan, debug)
                if next_search:
                    return next_search

    return []
    

In [495]:
for action in x.keys():
    z =possible_unification_iterator(x[action])
    for choice in z:
        print(action,choice)

use_on_self {('agent', '?agent'): ['agent', 'Me'], ('item', '?item'): ['item', 'Saw']}
use_on_self {('agent', '?agent'): ['agent', 'Other'], ('item', '?item'): ['item', 'Saw']}
use_on_self {('agent', '?agent'): ['agent', 'Me'], ('item', '?item'): ['item', 'Drill']}
use_on_self {('agent', '?agent'): ['agent', 'Other'], ('item', '?item'): ['item', 'Drill']}
use_on_other {('item', '?item'): ['item', 'Saw'], ('agent', '?agent1'): ['agent', 'Me'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Drill'], ('agent', '?agent1'): ['agent', 'Me'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Saw'], ('agent', '?agent1'): ['agent', 'Other'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Drill'], ('agent', '?agent1'): ['agent', 'Other'], ('agent', '?agent2'): ['agent', 'Me']}
use_on_other {('item', '?item'): ['item', 'Saw'], ('agent', '?agent1'): ['agent', 'Me'], ('agent', '?agent2'): ['agent',

### <a id="forward_planner"></a> forward_planner

Formal Parameters:

**start_state** a list of s-expressions representing the current state

**goal** the list of s-expressions representing the goal

**actions** the actions dictionary

**debug** A boolean -- Whether or not to include the intermediate states in plan

**return** The final **`plan`**, or `[]` if there can be no plan.


Calls [search](#search) to do the main work of the algorithm.

In [496]:


def forward_planner( start_state, goal, actions, debug=False):
    parse_simple(start_state)
    parse_simple(goal)
    parse_actions(actions)
    if debug:
        return search(start_state,goal,actions,[start_state],[start_state],debug)
    else:
        return search(start_state,goal,actions,[start_state],[],debug)
    
    #seen_states = {start_state:[]}
    
        
        
    return []

In [497]:
halloween_current_state_test1 = [
    "(item Saw)",
    "(item Drill)",
    "(agent Me)",
    "(agent Other)",
    "(at Me Earth)",
    "(at Other Earth)",
    "(Place Earth)",
    "(Place Narnia)",
    "(Place Purgatory)"
]

halloween_actions_test1 = {
    "use_on_self": {
        "action": "(use_on_self ?agent ?item)",
        "conditions": [
            "(agent ?agent)",
            "(item ?item)",
            "(at ?agent Earth)"
        ],
        "add": [
            "(at ?agent Narnia)"
        ],
        "delete": [
            "(at ?agent Earth)"
        ]
    },
    "use_on_other": {
        "action": "(use_on_other ?agent1 ?agent2 ?item)",
        "conditions": [
            "(item ?item)",
            "(agent ?agent1)",
            "(agent ?agent2)",
            "(at ?agent1 Earth)",
            "(at ?agent2 Earth)"
        ],
        "add": [
            "(at ?agent2 Narnia)",
            "(at ?agent1 Purgatory)"
        ],
        "delete": [
            "(at ?agent2 Earth)",
            "(at ?agent1 Earth)"
        ]
    }
}
halloween_goal_state_test1 = [
    "(item Saw)",
    "(item Drill)",
    "(agent Me)",
    "(agent Other)",
    "(at Other Purgatory)",
    "(at Me Narnia)",
    "(Place Earth)",
    "(Place Narnia)",
    "(Place Purgatory)"
]

test_plan = forward_planner( halloween_current_state_test1, halloween_goal_state_test1, halloween_actions_test1)
for el in test_plan:
    print(el)

['use_on_other', 'Other', 'Me', 'Saw']


In [498]:
halloween_current_state_test1 = [
    "(item Saw)",
    "(item Drill)",
    "(agent Me)",
    "(agent Other)",
    "(at Me Earth)",
    "(at Other Earth)",
    "(Place Earth)",
    "(Place Narnia)",
    "(Place Purgatory)"
]

halloween_actions_test1 = {
    "use_on_self": {
        "action": "(use_on_self ?agent ?item)",
        "conditions": [
            "(agent ?agent)",
            "(item ?item)",
            "(at ?agent Earth)"
        ],
        "add": [
            "(at ?agent Narnia)"
        ],
        "delete": [
            "(at ?agent Earth)"
        ]
    },
    "use_on_other": {
        "action": "(use_on_other ?agent1 ?agent2 ?item)",
        "conditions": [
            "(item ?item)",
            "(agent ?agent1)",
            "(agent ?agent2)",
            "(at ?agent1 Earth)",
            "(at ?agent2 Earth)"
        ],
        "add": [
            "(at ?agent2 Narnia)",
            "(at ?agent1 Purgatory)"
        ],
        "delete": [
            "(at ?agent2 Earth)",
            "(at ?agent1 Earth)"
        ]
    }
}
halloween_goal_state_test1 = [
    "(item Saw)",
    "(item Drill)",
    "(agent Me)",
    "(agent Other)",
    "(at Other Narnia)",
    "(at Me Narnia)",
    "(Place Earth)",
    "(Place Narnia)",
    "(Place Purgatory)"
]

test_plan = forward_planner( halloween_current_state_test1, halloween_goal_state_test1, halloween_actions_test1)
for el in test_plan:
    print(el)

['use_on_self', 'Me', 'Saw']
['use_on_self', 'Other', 'Saw']


You will be solving the problem from above. Here is the start state:

In [499]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [500]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [501]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

In [502]:
plan = forward_planner( start_state, goal, actions)

In [503]:
for el in plan:
    print(el)

['drive', 'Me', 'Home', 'Store']
['buy', 'Me', 'Store', 'Drill']
['drive', 'Me', 'Store', 'Home']


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.

For some reason, I cannot simply switch the arguments `debug=True` and `debug = False` in `plan = forward_planner( start_state, goal, actions)` without re-executing the entire notebook without getting an error.  I suspect this has something to do with how python treats default parameters (they are created in the global frame).  However, I've only encountered this problem before with objects, not booleans.  Perhaps it's jupyter notebook's fault, since I don't get this issue if I run my code in my IDE.

I was also unsure if I was implementing graph search or tree search correctly.  I assumed that graph search means that the seen states persist are only visited once, and never again, which is how I implemented the algorithm.